In [1]:
import argparse
import os
import torch
# 指定要设置的目标目录
target_directory = "/home/liq/sjw/ASE-COPY/hybrid_code/"
os.chdir(target_directory)
from utils import *
target_directory = "/home/liq/sjw/ASE-COPY/"
# 设置当前工作目录为指定目录
os.chdir(target_directory)
# 打印当前工作目录，以确认是否已经改变
print("当前工作目录:", os.getcwd())

/home/liq/.conda/envs/law/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


当前工作目录: /home/liq/sjw/ASE-COPY


In [2]:
def add_args(parser):
    # 预训练模型相关
    parser.add_argument("--model_type", default="unixCoder", type=str,
                        help="The model architecture to be fine-tuned.")
    parser.add_argument("--pretrain_dir", default="../transformers/unixcoder-base/", type=str,
                        help="The output directory where the model predictions and checkpoints will be written.")
    parser.add_argument("--lit_file", default="./dataset/intra_scenario_completion/java/literals.json", type=str,help="literals json file")

    # 待补全文件相关
    parser.add_argument("--data_dir", default=None, type=str,
                        help="The input data path.")
    parser.add_argument("--langs", default="java", type=str, 
                        help="Languages to train, if all, train all languages in data_dir")
    parser.add_argument("--output_dir", default=None, type=str, 
                        help="The output directory where the model predictions and checkpoints will be written.")
    
    # 检索相关
    parser.add_argument("--seed", default=42, type=int)
    parser.add_argument("--max_chunk_len", default=300, type=int)
    parser.add_argument("--block_k", default=10, type=int)
    parser.add_argument("--token_k", default=20, type=int)

    # 命令相关
    parser.add_argument("--data_process", action="store_true")
    parser.add_argument("--build_index", action='store_true')
    parser.add_argument("--do_search", action='store_true')
    parser.add_argument("--do_generate", action='store_true')
    parser.add_argument("--use_dense", action='store_true')
    parser.add_argument("--use_bm25", action='store_true')
    parser.add_argument("--use_hybrid", action='store_true')
    parser.add_argument("--bm_name", default="bm25", type=str, required=False, help="elasticsearch name.")
    parser.add_argument('--clearml_proj_name', type=str, default='')
    parser.add_argument('--log_file', type=str, default='log.log')


parser = argparse.ArgumentParser()
add_args(parser)
args = parser.parse_args(args=[])
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args.device = device
print(args)
special_tokens = get_special_tokens(args.lit_file)
tokenizer, model = load_pretrained_model(args, special_tokens)

model = model.to(device)
model.eval()
inputs = torch.load('cands.pth')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Namespace(model_type='unixCoder', pretrain_dir='../transformers/unixcoder-base/', lit_file='./dataset/intra_scenario_completion/java/literals.json', data_dir=None, langs='java', output_dir=None, seed=42, max_chunk_len=300, block_k=10, token_k=20, data_process=False, build_index=False, do_search=False, do_generate=False, use_dense=False, use_bm25=False, use_hybrid=False, bm_name='bm25', clearml_proj_name='', log_file='log.log', device=device(type='cuda', index=0))


In [3]:
from hybrid_code.knn_build import *
knn_saver = KNNSaver(dimension=model.config.hidden_size, pad_id=tokenizer.pad_token_id,
                                 only_errors=False)
knn_saver.break_into(model)

In [4]:
with torch.no_grad():
    model(inputs, labels=inputs)
    print(knn_saver.dstore_keys)

tensor([[ 0.0267, -0.0175, -0.1783,  ...,  0.7205,  1.5530, -0.6935],
        [ 0.0267, -0.0175, -0.1783,  ...,  0.7205,  1.5530, -0.6935],
        [ 0.0267, -0.0175, -0.1783,  ...,  0.7205,  1.5530, -0.6935],
        ...,
        [ 2.0536, -1.3802,  1.7088,  ...,  0.3719,  1.4268,  0.7988],
        [ 2.1275, -0.7531,  1.4362,  ...,  0.6786,  1.6657,  0.9966],
        [ 1.8327,  0.2165,  2.6935,  ..., -0.1264,  2.3622,  0.7895]],
       device='cuda:0')


In [5]:
knn_saver.break_out()
del knn_saver
from hybrid_master.knn_build import *
knn_saver = KNNSaver(dimension=model.config.hidden_size, pad_id=tokenizer.pad_token_id,
                                 only_errors=False)
knn_saver.break_into(model)

In [6]:
with torch.no_grad():
    model(inputs[:-1], labels=inputs[:-1])
    print(knn_saver.dstore_keys)

tensor([[ 0.0267, -0.0175, -0.1783,  ...,  0.7205,  1.5530, -0.6935],
        [ 0.0267, -0.0175, -0.1783,  ...,  0.7205,  1.5530, -0.6935],
        [ 0.0267, -0.0175, -0.1783,  ...,  0.7205,  1.5530, -0.6935],
        ...,
        [ 2.0536, -1.3802,  1.7088,  ...,  0.3719,  1.4268,  0.7988],
        [ 2.1275, -0.7531,  1.4362,  ...,  0.6786,  1.6657,  0.9966],
        [ 1.8327,  0.2165,  2.6935,  ..., -0.1264,  2.3622,  0.7895]],
       device='cuda:0')


In [2]:
print(output1['past_key_values'][0][0].shape)
past_key_values = list(output1['past_key_values'])
print(type(past_key_values))
for temp in range(len(past_key_values)):
    print(temp)
    print(len(past_key_values[temp]))
    print(past_key_values[temp][0].shape)
    print(past_key_values[temp][1].shape)
    print(past_key_values[temp][0][-1][:,:5,:].unsqueeze(0).shape)
    past_key_values[temp][0] = past_key_values[temp][0][-1][:,:5,:].unsqueeze(0)
    past_key_values[temp][1] = past_key_values[temp][1][-1][:,:5,:].unsqueeze(0)
import torch
logits = torch.load('logits.pth')
output_logits = torch.load('output_logits.pth')
queries = torch.load('queries.pth')
output_queries = torch.load('output_querys.pth')

In [3]:
import pickle
with open('test_past1.pkl', 'rb') as file:
    past1 = pickle.load(file)
with open('test_past2.pkl', 'rb') as file:
    past2 = pickle.load(file)